In [ ]:
import requests
import csv
import json
import socket 
from datetime import datetime

def get_block_info(block_hash):
    try:
        response = requests.get(f"https://blockchain.info/rawblock/{block_hash}")
        response.raise_for_status()
        return response.json()
    except requests.HTTPError as e:
        print(f"Ошибка HTTP: {e}")
        return None
    except Exception as e:
        print(f"Ошибка: {e}")
        return None

def check_blacklist(btc_address):
    try:
        hostname = f"{btc_address}.bl.btcblack.it"
        ip_address = socket.gethostbyname(hostname)
        return ip_address == "127.0.0.2"
    except socket.gaierror:
        return False

def is_suspicious(addreses):
    for adress in addreses:
        if check_blacklist(adress):
            return True
    return False

def process_transactions(transactions, csv_writer):
    for tx in transactions:
        inputs = {inp["prev_out"]["addr"]: inp["prev_out"]["value"] / 100000000 for inp in tx["inputs"] if "prev_out" in inp and "addr" in inp["prev_out"]}
        outputs = {out["addr"]: out["value"] / 100000000 for out in tx["out"] if "addr" in out}
        total_amount = sum(outputs.values())
        suspicious = is_suspicious(list(inputs.keys()) + list(outputs.keys()))
        tx_details = {
            "Timestamp": (tx["time"]),
            "TotalAmount": total_amount,
            "TotalInputs": len(inputs),
            "TotalOutputs": len(outputs),
            "Inputs": json.dumps(inputs),
            "Outputs": json.dumps(outputs),
            "Suspicius": suspicious,
        }
        csv_writer.writerow(tx_details)

def fetch_transactions_and_write_csv(last_n_blocks=80, csv_filename="transactions.csv"):
    try:
        latest_block_hash = requests.get("https://blockchain.info/latestblock").json()["hash"]

        with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
            fieldnames = ["Timestamp", "TotalAmount", "TotalInputs", "TotalOutputs", "Inputs", "Outputs", "Suspicius"]
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            
            for _ in range(last_n_blocks):
                block_data = get_block_info(latest_block_hash)
                if block_data:
                    process_transactions(block_data["tx"], writer)
                    latest_block_hash = block_data["prev_block"]
                else:
                    print("Ошибка получения данных о блоке.")
                    break
    except Exception as e:
        print(f"Ошибка: {e}")

fetch_transactions_and_write_csv()


<div style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; color: #333;">
  <h2 style="color: #2E8B57; font-family: 'Georgia'; border-bottom: 2px solid #2E8B57; padding-bottom: 4px;">Описание скрипта анализа транзакций Bitcoin</h2>

  <p><strong style="color: #8B0000; text-decoration: underline;">Импорт необходимых библиотек:</strong> В начале скрипта импортируются библиотеки requests для работы с HTTP запросами, csv для работы с CSV файлами, json для обработки JSON данных, socket для DNS запросов, и datetime для работы с датами и временем.</p>

  <h3 style="color: #3A5FCD; font-family: 'Arial'; margin-top: 24px;">Функции скрипта:</h3>

  <ol style="margin-left: 20px; list-style-type: square; font-size: 16px; line-height: 1.6;">
    <li><strong style="color: #2F4F4F;">get_block_info(block_hash):</strong> Функция принимает хеш блока и возвращает информацию о блоке, используя API blockchain.info. В случае ошибок в HTTP запросе, функция выводит сообщение об ошибке.</li>
    <li><strong style="color: #2F4F4F;">check_blacklist(btc_address):</strong> Эта функция проверяет, находится ли биткойн-адрес в чёрном списке, осуществляя DNS запрос к btcblack.it. Возвращает True, если адрес находится в чёрном списке.</li>
    <li><strong style="color: #2F4F4F;">is_suspicious(addresses):</strong> Функция принимает список биткойн-адресов и проверяет, находится ли хотя бы один из них в чёрном списке, используя функцию check_blacklist.</li>
    <li><strong style="color: #2F4F4F;">process_transactions(transactions, csv_writer):</strong> Эта функция обрабатывает список транзакций, создавая словарь с деталями каждой транзакции, включая информацию о подозрительности транзакции. Затем записывает эти данные в CSV файл.</li>
  </ol>

  <h3 style="color: #3A5FCD; font-family: 'Arial'; margin-top: 24px;">Основной цикл скрипта:</h3>

  <p style="font-size: 16px; line-height: 1.6;">В основной части скрипта выполняются следующие действия:</p>

  <ol style="margin-left: 20px; list-style-type: square; font-size: 16px; line-height: 1.6;">
    <li>Получение хеша последнего блока из блокчейна Bitcoin.</li>
    <li>Открытие файла CSV для записи данных и инициализация CSV writer.</li>
    <li>Циклический проход по последним 80 блокам, получение данных о каждом блоке и обработка транзакций в них с помощью функции process_transactions.</li>
    <li>Запись информации о каждой транзакции в CSV файл с учетом того, является ли транзакция подозрительной.</li>
  </ol>

  <p style="font-size: 16px; line-height: 1.6;">Скрипт завершает свою работу после обработки указанного количества блоков или в случае возникновения ошибки.</p>
</div>


In [ ]:
import pandas as pd
import json
import numpy as np
import socket 
from datetime import datetime

df = pd.read_csv('transactions.csv')

wallets = {}

def check_blacklist(btc_address):
    try:
        hostname = f"{btc_address}.bl.btcblack.it"
        ip_address = socket.gethostbyname(hostname)
        return ip_address == "127.0.0.2"
    except socket.gaierror:
        return False

# Функция для обновления агрегированных данных кошелька
def update_wallet(wallet, amount, is_input, inputs, outputs, row):
    timestamp = datetime.fromtimestamp(row['Timestamp'])
    if wallet not in wallets:
        wallets[wallet] = {'total_sent': 0, 'total_received': 0, 'tx_amounts': [],
                           'tx_timestamps': [], 'unique_addresses': set(),
                           'total_inputs': 0, 'total_outputs': 0, 'tx_counts': 0,
                           'blacklisted': check_blacklist(wallet)}
    
    if is_input:
        wallets[wallet]['total_received'] += amount
    else:
        wallets[wallet]['total_sent'] += amount

    wallets[wallet]['tx_amounts'].append(amount)
    wallets[wallet]['tx_timestamps'].append(timestamp)
    wallets[wallet]['unique_addresses'].update(inputs.keys())
    wallets[wallet]['unique_addresses'].update(outputs.keys())
    wallets[wallet]['total_inputs'] += row['TotalInputs']
    wallets[wallet]['total_outputs'] += row['TotalOutputs']
    wallets[wallet]['tx_counts'] += 1

# Обработка каждой строки в датасете
for index, row in df.iterrows():
    inputs = json.loads(row['Inputs'])
    outputs = json.loads(row['Outputs'])
    total_amount = row['TotalAmount']
    
    for wallet in inputs:
        update_wallet(wallet, total_amount, True, inputs, outputs, row)

    for wallet in outputs:
        update_wallet(wallet, total_amount, False, inputs, outputs, row)

# Расчет интервалов между транзакциями
def calculate_intervals(timestamps):
    timestamps.sort()
    intervals = [(timestamps[i] - timestamps[i-1]).total_seconds() for i in range(1, len(timestamps))]
    return intervals if intervals else [0]

# Преобразование словаря в DataFrame и расчет дополнительных признаков
wallets_df = pd.DataFrame.from_dict(wallets, orient='index')
wallets_df['wallet'] = wallets_df.index
wallets_df['unique_address_count'] = wallets_df['unique_addresses'].apply(len)
wallets_df['avg_interval_between_tx'] = wallets_df['tx_timestamps'].apply(calculate_intervals).apply(np.mean)
wallets_df['avg_tx_amount'] = wallets_df['tx_amounts'].apply(np.mean)
wallets_df['max_tx_amount'] = wallets_df['tx_amounts'].apply(max)
wallets_df['min_tx_amount'] = wallets_df['tx_amounts'].apply(min)
wallets_df['avg_inputs'] = wallets_df['total_inputs'] / wallets_df['tx_counts']
wallets_df['avg_outputs'] = wallets_df['total_outputs'] / wallets_df['tx_counts']

# Очистка от временных колонок
wallets_df.drop(columns=['tx_amounts', 'tx_timestamps', 'unique_addresses', 'total_inputs', 'total_outputs', 'tx_counts'], inplace=True)

# Сохранение обработанного датасета в файл CSV
wallets_df.to_csv('wallets.csv', index=False)

print("Датасет кошельков сохранен в файл 'wallets.csv'.")

<div style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; color: #333;">
  <h2 style="color: #2E8B57; font-family: 'Georgia'; border-bottom: 2px solid #2E8B57; padding-bottom: 4px;">Обработка и Анализ Данных Кошельков Bitcoin</h2>

  <p><strong style="color: #8B0000; text-decoration: underline;">Импорт библиотек:</strong> Скрипт начинается с импорта необходимых модулей для обработки данных, сетевых запросов и работы со временем.</p>

  <h3 style="color: #3A5FCD; font-family: 'Arial'; margin-top: 24px;">Основные функции:</h3>

  <ol style="margin-left: 20px; list-style-type: square; font-size: 16px; line-height: 1.6;">
    <li><strong style="color: #2F4F4F;">check_blacklist(btc_address):</strong> Проверяет биткойн-адрес на наличие в DNS-черном списке, что может указывать на его участие в подозрительной деятельности.</li>
    <li><strong style="color: #2F4F4F;">update_wallet(wallet, amount, is_input, inputs, outputs, row):</strong> Обновляет агрегированные данные о кошельке, учитывая новые транзакции, вводит флаг черного списка, и добавляет данные в общий словарь.</li>
    <li><strong style="color: #2F4F4F;">calculate_intervals(timestamps):</strong> Рассчитывает интервалы времени между последовательными транзакциями для каждого кошелька.</li>
  </ol>

  <h3 style="color: #3A5FCD; font-family: 'Arial'; margin-top: 24px;">Процесс анализа данных:</h3>

  <ol style="margin-left: 20px; list-style-type: square; font-size: 16px; line-height: 1.6;">
    <li>Загрузка данных из CSV файла с транзакциями.</li>
    <li>Итерация по каждой транзакции и ее компонентам (inputs и outputs), анализ и обновление информации о кошельках.</li>
    <li>Применение функций для расчета интервалов и агрегирования данных, таких как средние значения и счетчики транзакций.</li>
    <li>Создание окончательного DataFrame с обновленной информацией по каждому кошельку и сохранение его в CSV файл для дальнейшего использования.</li>
  </ol>

  <p style="font-size: 16px; line-height: 1.6;">В конце выполнения, данные сохраняются в файл 'wallets.csv', и скрипт выводит соответствующее уведомление.</p>
</div>


<div style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; color: #333;">
  <h2 style="color: #2E8B57; font-family: 'Georgia'; margin-bottom: 4px;">Параметры Анализа Кошельков</h2>

  <p><strong style="color: #8B0000;">Уникальное количество адресов, с которыми проводилась операция:</strong> Этот параметр позволяет оценить разнообразие взаимодействий кошелька с другими участниками сети. Чем больше уникальных адресов, тем шире сеть контактов кошелька, что может указывать на повышенную активность или коммерческое использование.</p>

  <p><strong style="color: #8B0000;">Средний интервал между транзакциями:</strong> Показывает, насколько часто кошелек совершает транзакции. Короткие интервалы могут указывать на высокую активность кошелька, в то время как длинные интервалы могут свидетельствовать о периодическом или редком использовании.</p>

  <p><strong style="color: #8B0000;">Средний размер транзакции:</strong> Это измерение позволяет оценить обычный объем средств, перемещаемых через кошелек. Он может выявить, например, отличаются ли операции данного кошелька от среднестатистических, что может быть признаком необычной активности.</p>

  <p><strong style="color: #8B0000;">Максимальный размер транзакции:</strong> Предоставляет информацию о самой крупной транзакции, проведенной через кошелек. Это может быть важным признаком, особенно если кошелек используется для крупных, возможно, подозрительных операций.</p>

  <p><strong style="color: #8B0000;">Минимальный размер транзакции:</strong> Отражает наименьшую сумму, переданную через кошелек. Это может помочь выявить кошельки, используемые для мелких транзакций или для операций с низкой стоимостью.</p>

  <p><strong style="color: #8B0000;">Среднее количество inputs в транзакциях с этим кошельком:</strong> Отражает типичное количество входящих связей в транзакциях. Большое число входов может указывать на сбор средств с множества источников, что характерно для некоторых типов кошельков или схем.</p>

  <p><strong style="color: #8B0000;">Среднее количество outputs в транзакциях с этим кошельком:</strong> Показывает, как много адресов обычно получает средства в результате транзакций с данным кошельком. Это может быть признаком распределения средств между множеством участников.</p>

  <p><strong style="color: #8B0000;">Есть ли кошелек в блеклисте:</strong> Указывает, был ли кошелек замечен в подозрительной или незаконной деятельности. Наличие кошелька в черном списке может служить основанием для дополнительного расследования или мониторинга его активности.</p>
</div>


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

df = pd.read_csv('transactions.csv')
df.drop(columns=['Inputs', 'Outputs', 'Timestamp'], inplace=True)
df['TotalInputs'] = df['TotalInputs'].astype(float)
df['TotalOutputs'] = df['TotalOutputs'].astype(float)

X = df.drop('Suspicius', axis=1)
y = df['Suspicius']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

xgb_model = XGBClassifier(random_state=42)
parameters = {'n_estimators': [100, 200, 300], 'max_depth': [3, 5, 7], 'learning_rate': [0.01, 0.1, 0.2]}
grid_search = GridSearchCV(xgb_model, parameters, cv=3, scoring='accuracy')
grid_search.fit(X_train_balanced, y_train_balanced)

best_model = grid_search.best_estimator_
best_model.fit(X_train_balanced, y_train_balanced)

predictions = best_model.predict(X_test)
print(classification_report(y_test, predictions))



<div style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; color: #333;">
  <h2 style="color: #2E8B57; font-family: 'Georgia'; border-bottom: 2px solid #2E8B57; padding-bottom: 4px;">Описание скрипта машинного обучения для анализа транзакций Bitcoin</h2>

  <p><strong style="color: #8B0000; text-decoration: underline;">Импорт библиотек и подготовка данных:</strong> В начале скрипта загружаем необходимые библиотеки и подготавливаем данные, удаляя нерелевантные столбцы и преобразуя данные для анализа.</p>

  <h3 style="color: #3A5FCD; font-family: 'Arial'; margin-top: 24px;">Этапы обработки данных и подготовки модели:</h3>

  <ol style="margin-left: 20px; list-style-type: square; font-size: 16px; line-height: 1.6;">
    <li><strong style="color: #2F4F4F;">Предобработка данных:</strong> Удаление столбцов 'Inputs', 'Outputs', 'Timestamp' и преобразование оставшихся данных для анализа.</li>
    <li><strong style="color: #2F4F4F;">Разделение данных:</strong> Разбиение набора данных на обучающую и тестовую выборки.</li>
    <li><strong style="color: #2F4F4F;">Балансировка классов:</strong> Применение метода SMOTE для устранения дисбаланса классов в обучающей выборке.</li>
    <li><strong style="color: #2F4F4F;">Настройка и обучение модели:</strong> Использование XGBoost с настройкой гиперпараметров через GridSearchCV для нахождения оптимальной модели.</li>
    <li><strong style="color: #2F4F4F;">Оценка модели:</strong> Оценка качества работы лучшей модели на тестовых данных.</li>
  </ol>

  <h3 style="color: #3A5FCD; font-family: 'Arial'; margin-top: 24px;">Особенности и нюансы реализации:</h3>

  <ul style="margin-left: 20px; list-style-type: circle; font-size: 16px; line-height: 1.6;">
    <li>Скрипт акцентирует внимание на качественном анализе подозрительных транзакций.</li>
    <li>Применение перебалансировки классов и тщательный подбор гиперпараметров помогают в увеличении точности модели.</li>
    <li>Работа скрипта может занять продолжительное время из-за обширного поиска параметров в GridSearchCV.</li>
  </ul>

  <p style="font-size: 16px; line-height: 1.6;">В целом, скрипт представляет собой комплексный подход к обучению и оценке машинной модели для выявления аномалий в блокчейн-транзакциях.</p>
</div>


<h2>Анализ результатов модели машинного обучения</h2>
<p>Модель машинного обучения была оценена на основе различных метрик производительности, подробности которых представлены ниже.</p>
    
<h3>Оценка для класса 'False' (Не подозрительные транзакции):</h3>
<ul>
    <li><strong>Precision:</strong> 0.99 - доля правильно классифицированных не подозрительных транзакций от всех классифицированных как не подозрительные.</li>
    <li><strong>Recall:</strong> 0.84 - доля правильно идентифицированных не подозрительных транзакций от всех реально не подозрительных.</li>
    <li><strong>F1-score:</strong> 0.91 - баланс между точностью и полнотой для не подозрительных транзакций.</li>
</ul>
    
<h3>Оценка для класса 'True' (Подозрительные транзакции):</h3>
<ul>
    <li><strong>Precision:</strong> 0.03 - доля правильно классифицированных подозрительных транзакций от всех классифицированных как подозрительные.</li>
    <li><strong>Recall:</strong> 0.47 - доля правильно идентифицированных подозрительных транзакций от всех реально подозрительных.</li>
    <li><strong>F1-score:</strong> 0.05 - показатель эффективности модели при классификации подозрительных транзакций.</li>
</ul>

<h3>Общие метрики:</h3>
<ul>
    <li><strong>Accuracy:</strong> 0.84 - общий процент правильных предсказаний модели.</li>
    <li><strong>Macro avg:</strong> Примерно 0.51 и 0.48 для точности и F1-score соответственно, что указывает на средний уровень производительности модели.</li>
    <li><strong>Weighted avg:</strong> Примерно 0.99 и 0.90 для точности и F1-score соответственно, что лучше из-за большего количества примеров в классе 'False'.</li>
</ul>

<p>Вывод: модель хорошо идентифицирует не подозрительные транзакции, но имеет низкую точность в определении подозрительных транзакций. Сильный дисбаланс классов может вносить вклад в эти результаты.</p>
